In [13]:
!git clone https://github.com/sahithi-r/TextSummarizer.git

Cloning into 'TextSummarizer'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 77 (delta 37), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (77/77), 27.23 KiB | 13.61 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [14]:
import os
os.chdir("TextSummarizer")

In [15]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=c77298db2f0773b253ed0f822184a9c9a6ae276c1de847149c82db3963cae143
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ngrok authtoken 2fCAa6Oh2nEd9sUm55B5b5tJVU6_ZnYRbid3KsDk9AsiFSsF

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# Fine-tuning using the Bart Base Model

### Loading Dataset and preprocessing the train, test and validation data from the CNN Dailymail dataset using Bart-base tokenizer

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load pre-trained model and tokenizer
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Load news dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]

    # Tokenize inputs with truncation and padding
    inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets with truncation and padding
    targets = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    model_inputs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

    return model_inputs

dataset_train = dataset["train"].select(range(10000)).map(preprocess_function, batched=True)
dataset_val = dataset["validation"].select(range(1000)).map(preprocess_function, batched=True)
dataset_test = dataset["test"].select(range(1000)).map(preprocess_function, batched=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Saving dataset to drive

In [ ]:
dataset_train.save_to_disk("/content/drive/My Drive/10k-train-bart-base")
dataset_val.save_to_disk("/content/drive/My Drive/1k-val-bart-base")
dataset_test.save_to_disk("/content/drive/My Drive/1k-test-bart-base")

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

### Fine-tuning the model after setting training arguments



In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results-bart",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=12,
    num_train_epochs=4,
    logging_dir="./logs-bart",
)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
)

# Fine-tune the model
trainer.train()

Step,Training Loss


### Saving model to drive

In [ ]:
model.save_pretrained("/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K")
tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/vocab.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/merges.txt',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/added_tokens.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K/tokenizer.json')

# Loading saved model and dataset ( without running above cells )

In [39]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BartForConditionalGeneration,  AutoModelForSeq2SeqLM
from datasets import load_from_disk

model_path = "/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-bart-10K"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the fine-tuned model
model =  AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

train_dataset_path = "/content/drive/My Drive/10k-train-bart-base"
val_dataset_path = "/content/drive/My Drive/1k-val-bart-base"
test_dataset_path = "/content/drive/My Drive/1k-test-bart-base"

# Load the datasets
dataset_train = load_from_disk(train_dataset_path)
dataset_val = load_from_disk(val_dataset_path)
dataset_test = load_from_disk(test_dataset_path)


## Running Rouge Metric on 10 entries from test set

In [40]:
from rouge import Rouge

rouge = Rouge()

for entry in dataset_test.select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    model = model.to(device)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)


Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

## Testing on Bart Base model

In [41]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_from_disk

model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

train_dataset_path = "/content/drive/My Drive/10k-train-bart-base"
val_dataset_path = "/content/drive/My Drive/1k-val-bart-base"
test_dataset_path = "/content/drive/My Drive/1k-test-bart-base"

# Load the datasets
dataset_train_t5 = load_from_disk(train_dataset_path)
dataset_val_t5 = load_from_disk(val_dataset_path)
dataset_test_t5 = load_from_disk(test_dataset_path)

rouge = Rouge()
for entry in dataset_test_t5.select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    model = model.to(device)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)



Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

# Fine-tuning using the T5 Small Model

### Loading Dataset and preprocessing the train, test and validation data from the CNN Dailymail dataset using Bart-base tokenizer

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [ ]:
MODEL = 't5-small'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 4
OUT_DIR = 'results_t5small'
MAX_LENGTH = 512
tokenizer = T5Tokenizer.from_pretrained(MODEL)

# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['article']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['highlights']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset_train_t5 = dataset["train"].select(range(10000)).map(preprocess_function, batched=True)
dataset_val_t5 = dataset["validation"].select(range(1000)).map(preprocess_function, batched=True)
dataset_test_t5 = dataset["test"].select(range(1000)).map(preprocess_function, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Saving dataset to Drive

In [ ]:
dataset_train_t5.save_to_disk("/content/drive/My Drive/10k-train-t5")
dataset_val_t5.save_to_disk("/content/drive/My Drive/1k-val-t5")
dataset_test_t5.save_to_disk("/content/drive/My Drive/1k-test-t5")

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

### Pre-processing data with tokenizer and setting up rouge metric for fine-tuning

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):

    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

### Fine-tuning the model after setting training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_t5,
    eval_dataset=dataset_val_t5,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

history = trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
200,0.353000,0.250425,0.512100,0.251000,0.472600,49.160000
400,0.296800,0.214586,0.523000,0.260100,0.482100,49.624000
600,0.308800,0.213823,0.522100,0.257600,0.480100,50.158000
800,0.271800,0.213450,0.521700,0.259400,0.480300,50.184000
1000,0.288100,0.212556,0.521400,0.259500,0.479800,50.188000
1200,0.302300,0.211780,0.522700,0.260200,0.480700,50.189000
1400,0.268900,0.212714,0.520800,0.258800,0.479300,50.190000
1600,0.260100,0.213350,0.524500,0.261300,0.481100,50.191000
1800,0.251300,0.213312,0.521300,0.259800,0.479700,50.191000
2000,0.243400,0.213073,0.519500,0.257600,0.478100,50.191000


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


### Save model to Drive

In [ ]:
model.save_pretrained("/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K")
tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K")

('/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K/spiece.model',
 '/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K/added_tokens.json')

# Loading saved model and dataset ( without running above cells )

In [37]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_from_disk

model_path = "/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K"

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load the fine-tuned model
model =  T5ForConditionalGeneration.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

train_dataset_path = "/content/drive/My Drive/10k-train-t5"
val_dataset_path = "/content/drive/My Drive/1k-val-t5"
test_dataset_path = "/content/drive/My Drive/1k-test-t5"

# Load the datasets
dataset_train_t5 = load_from_disk(train_dataset_path)
dataset_val_t5 = load_from_disk(val_dataset_path)
dataset_test_t5 = load_from_disk(test_dataset_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Running Rouge Metric on 10 entries from test set

In [38]:
rouge = Rouge()
for entry in dataset_test_t5.select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    model = model.to(device)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)


Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

## Testing on T5 Small model

In [36]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_from_disk

model_path = "t5-small"

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load the fine-tuned model
model =  T5ForConditionalGeneration.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

train_dataset_path = "/content/drive/My Drive/10k-train-t5"
val_dataset_path = "/content/drive/My Drive/1k-val-t5"
test_dataset_path = "/content/drive/My Drive/1k-test-t5"

# Load the datasets
dataset_train_t5 = load_from_disk(train_dataset_path)
dataset_val_t5 = load_from_disk(val_dataset_path)
dataset_test_t5 = load_from_disk(test_dataset_path)

rouge = Rouge()
for entry in dataset_test_t5.select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    model = model.to(device)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

# Using a pretrained model on CNN-Dailymail- Distilbart

In [42]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_from_disk
from rouge import Rouge

model_path = "sshleifer/distilbart-cnn-12-6"
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

# Load news dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]

    # Tokenize inputs with truncation and padding
    inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets with truncation and padding
    targets = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    model_inputs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"],
    }

    return model_inputs

dataset_train = dataset["train"].select(range(10000)).map(preprocess_function, batched=True)
dataset_val = dataset["validation"].select(range(1000)).map(preprocess_function, batched=True)
dataset_test = dataset["test"].select(range(1000)).map(preprocess_function, batched=True)

rouge = Rouge()

for entry in dataset_test.select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]

    # Tokenize the article
    inputs = tokenizer(article, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
    model_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

# Extractive Summarization - Using Class Rank based on TF-IDF

In [51]:
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
nltk.download('punkt')

def get_extractive_summary(text):
  num_sentences = 3
  sentences = sent_tokenize(text)
  tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]
  preprocessed_sentences = [" ".join(words) for words in tokenized_sentences]
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(preprocessed_sentences)
  similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

  scores = np.sum(similarity_matrix, axis=1)
  ranked_sentences = sorted(((scores[i], i) for i in range(len(sentences))), reverse=True)
  top_sentences = sorted(ranked_sentences[:num_sentences], key=lambda x: x[1])

  summary = ' '.join([sentences[idx] for score, idx in top_sentences])
  return summary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
rouge = Rouge()

for entry in dataset['test'].select(range(10)):
    article = entry["article"]
    reference_summary = entry["highlights"]

    # Tokenize the article
    model_summary = get_extractive_summary(article)
    scores = rouge.get_scores(model_summary, reference_summary)
    print("Article:\n", article)
    print("Reference Summary:\n", reference_summary)
    print("Generated Summary:\n", model_summary)
    print("ROUGE Scores:", scores)
    print("="*50)

Article:
 (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ce

##**Final Implementation of Backend code - With Extractive and Abstractive Summarization**

In [ ]:
from pyngrok import ngrok
from flask import Flask, render_template, request, jsonify
from transformers import pipeline
import numpy as np
import networkx as nx
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
model_path = "/content/drive/My Drive/fine_tuned_model_on_dailymail-cnn-t5s-10K"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model =  T5ForConditionalGeneration.from_pretrained(model_path)


app = Flask(__name__)

public_url = ngrok.connect()
print(' * ngrok tunnel:', public_url)

@app.route('/')
def initial():
    return render_template('index.html')

@app.route('/submit-text-abs', methods=['POST'])
def generate_summary_abs():
    prompt = request.form['summarizationInput']
    print(f"Generating summary of: {prompt}")
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=128, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(summary)
    return summary


@app.route('/submit-text-ext', methods=['POST'])
def generate_summary_ext():
  num_sentences=3
  text = request.form['summarizationInput']

  sentences = sent_tokenize(text)

  tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]
  preprocessed_sentences = [" ".join(words) for words in tokenized_sentences]
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(preprocessed_sentences)
  similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

  scores = np.sum(similarity_matrix, axis=1)
  ranked_sentences = sorted(((scores[i], i) for i in range(len(sentences))), reverse=True)
  top_sentences = sorted(ranked_sentences[:num_sentences], key=lambda x: x[1])

  summary = ' '.join([sentences[idx] for score, idx in top_sentences])

  return summary

if __name__ == '__main__':
    app.run(port=80)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 * ngrok tunnel: http://3e54-34-125-187-94.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:80
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Apr/2024 00:00:02] "GET / HTTP/1.1" 200 -


Generating summary of: 
Britain’s King Charles III will resume public duties next week following “a period of treatment and recuperation,” Buckingham Palace announced Friday, two months after revealing that he was being treated for cancer.

The 75-year-old monarch will be joined by his wife, Queen Camilla, for a visit to a cancer treatment center on Tuesday “where they will meet medical specialists and patients,” according to a palace statement.

It is understood that the facility is not the same treatment center that has been directly involved in the King’s medical care.

It will be the first of several engagements the King is expected to carry out in the weeks ahead. The palace also announced that the British sovereign would host Japanese Emperor Naruhito and Empress Masako for a state visit in June.

“As the first anniversary of the Coronation approaches, Their Majesties remain deeply grateful for the many kindnesses and good wishes they have received from around the world throughou

INFO:werkzeug:127.0.0.1 - - [28/Apr/2024 00:00:09] "POST /submit-text-abs HTTP/1.1" 200 -


Britain’s King Charles III will resume public duties next week. The 75-year-old monarch will be joined by his wife, Queen Camilla, for a visit to a cancer treatment center. The British sovereign will host Japanese Emperor Naruhito and Empress Masako for a state visit in June.
